In [1]:
import os
from os import listdir
from os.path import join, isfile
from PIL import Image

import numpy as np
import pandas as pd
# import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from efficientnet_pytorch import EfficientNet
from torchvision.transforms import transforms, Lambda
from torch.utils.data import random_split
import random
from torch.utils.data import DataLoader
import glob

In [59]:
original_address = '/scratch/student/shokoofa/datasets'

In [60]:
info = pd.read_csv('final_large_labels.csv')

In [61]:
info

,path,label,start,end
0,6fwt32TiT_KD-QU8t64_eg/65,Apical-3ch,2,17
1,6fwt32TiT_KD-QU8t64_eg/65,Apical-3ch,4,19
2,6fwt32TiT_KD-QU8t64_eg/65,Apical-3ch,6,21
3,6fwt32TiT_KD-QU8t64_eg/65,Apical-3ch,8,23
4,6fwt32TiT_KD-QU8t64_eg/65,Apical-3ch,10,25
...,...,...,...,...
35388,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,13,28
35389,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,15,30
35390,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,17,32
35391,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,19,34


In [62]:
required_labels = ['PSAX-ves', 'PSAX-base', 'PSAX-mid', 'PSAX-apical']

for i, row in enumerate(info['label']):
    if row not in required_labels:
        info.drop(i, axis=0, inplace= True)

In [63]:
info['patient'] = info['path'].str.split('/').str[0]

In [64]:
grouped = info.groupby('patient')
c = 0
for patient, group in grouped:
    print(f"Patient: {patient}")
    print(group, "\n")
    c += 1
print(c)

Patient: -40uXj7VSHK4KVuHfAjiYQ
                            path     label  start  end                 patient
13831  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves      5   20  -40uXj7VSHK4KVuHfAjiYQ
13832  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves      7   22  -40uXj7VSHK4KVuHfAjiYQ
13833  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves      9   24  -40uXj7VSHK4KVuHfAjiYQ
13834  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves     11   26  -40uXj7VSHK4KVuHfAjiYQ
13835  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves     13   28  -40uXj7VSHK4KVuHfAjiYQ
...                          ...       ...    ...  ...                     ...
14156  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     47   62  -40uXj7VSHK4KVuHfAjiYQ
14157  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     49   64  -40uXj7VSHK4KVuHfAjiYQ
14158  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     51   66  -40uXj7VSHK4KVuHfAjiYQ
14159  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     53   68  -40uXj7VSHK4KVuHfAjiYQ
14160  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     55   70  -40uXj7VSHK4KVuHfAjiYQ

[149 rows x 5 colum

In [65]:
patient_dataframes = {patient: group for patient, group in grouped}

In [66]:
patient_dataframes

{'-40uXj7VSHK4KVuHfAjiYQ':                             path     label  start  end                 patient
 13831  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves      5   20  -40uXj7VSHK4KVuHfAjiYQ
 13832  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves      7   22  -40uXj7VSHK4KVuHfAjiYQ
 13833  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves      9   24  -40uXj7VSHK4KVuHfAjiYQ
 13834  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves     11   26  -40uXj7VSHK4KVuHfAjiYQ
 13835  -40uXj7VSHK4KVuHfAjiYQ/21  PSAX-ves     13   28  -40uXj7VSHK4KVuHfAjiYQ
 ...                          ...       ...    ...  ...                     ...
 14156  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     47   62  -40uXj7VSHK4KVuHfAjiYQ
 14157  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     49   64  -40uXj7VSHK4KVuHfAjiYQ
 14158  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     51   66  -40uXj7VSHK4KVuHfAjiYQ
 14159  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     53   68  -40uXj7VSHK4KVuHfAjiYQ
 14160  -40uXj7VSHK4KVuHfAjiYQ/40  PSAX-ves     55   70  -40uXj7VSHK4KVuHfAjiYQ
 
 [149 rows x

In [80]:
import numpy as np

# Unique patients
unique_patients = info['patient'].unique()

# Shuffling to randomize the patient order
np.random.shuffle(unique_patients)

split_val_index = int(len(unique_patients) * 0.8)
split_test_index = int(len(unique_patients) - split_val_index * 0.1)

train_patients = unique_patients[:split_val_index]
val_patients = unique_patients[split_val_index:split_test_index]
test_patients = unique_patients[split_test_index:]


train_data = info[info['patient'].isin(train_patients)].reset_index()
val_data = info[info['patient'].isin(val_patients)].reset_index()
test_data = info[info['patient'].isin(test_patients)].reset_index()

In [81]:
train_data

,index,path,label,start,end,patient
0,871,6VLObFIMTzq7fyUz9-ltQA/65,PSAX-base,1,16,6VLObFIMTzq7fyUz9-ltQA
1,872,6VLObFIMTzq7fyUz9-ltQA/65,PSAX-base,3,18,6VLObFIMTzq7fyUz9-ltQA
2,873,6VLObFIMTzq7fyUz9-ltQA/65,PSAX-base,5,20,6VLObFIMTzq7fyUz9-ltQA
3,874,6VLObFIMTzq7fyUz9-ltQA/65,PSAX-base,7,22,6VLObFIMTzq7fyUz9-ltQA
4,875,6VLObFIMTzq7fyUz9-ltQA/65,PSAX-base,9,24,6VLObFIMTzq7fyUz9-ltQA
...,...,...,...,...,...,...
8825,35388,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,13,28,3nRPNSozRYSJy2tmnGdyzg
8826,35389,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,15,30,3nRPNSozRYSJy2tmnGdyzg
8827,35390,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,17,32,3nRPNSozRYSJy2tmnGdyzg
8828,35391,3nRPNSozRYSJy2tmnGdyzg/29,PSAX-apical,19,34,3nRPNSozRYSJy2tmnGdyzg


In [82]:
test_data

,index,path,label,start,end,patient
0,510,5CH7tKU5QJOmQxevSWzZDQ/30,PSAX-mid,2,17,5CH7tKU5QJOmQxevSWzZDQ
1,511,5CH7tKU5QJOmQxevSWzZDQ/30,PSAX-mid,4,19,5CH7tKU5QJOmQxevSWzZDQ
2,512,5CH7tKU5QJOmQxevSWzZDQ/30,PSAX-mid,6,21,5CH7tKU5QJOmQxevSWzZDQ
3,513,5CH7tKU5QJOmQxevSWzZDQ/30,PSAX-mid,8,23,5CH7tKU5QJOmQxevSWzZDQ
4,514,5CH7tKU5QJOmQxevSWzZDQ/30,PSAX-mid,10,25,5CH7tKU5QJOmQxevSWzZDQ
...,...,...,...,...,...,...
932,31696,_kXOP1MSTci8eNMele9Atw/19,PSAX-mid,22,37,_kXOP1MSTci8eNMele9Atw
933,31697,_kXOP1MSTci8eNMele9Atw/19,PSAX-mid,24,39,_kXOP1MSTci8eNMele9Atw
934,31698,_kXOP1MSTci8eNMele9Atw/19,PSAX-mid,26,41,_kXOP1MSTci8eNMele9Atw
935,31699,_kXOP1MSTci8eNMele9Atw/19,PSAX-mid,28,43,_kXOP1MSTci8eNMele9Atw


In [83]:
import shutil 
# os.makedirs(os.path.join(original_address, '/Train'), exist_ok=True)
# os.makedirs(os.path.join(original_address, '/Test'), exist_ok=True)
# os.makedirs(os.path.join(original_address, '/Val'), exist_ok=True)

for path in train_data['path'].unique():
    source_path = os.path.join(original_address+ '/', path)
    destination_path = os.path.join(original_address +  '/Train/', path)
    # print(source_path, destination_path)
    shutil.move(source_path, destination_path)

In [84]:
for path in test_data['path'].unique():
    source_path = os.path.join(original_address+ '/', path)
    destination_path = os.path.join(original_address +  '/Test/', path)
    shutil.move(source_path, destination_path)

In [85]:
for path in val_data['path'].unique():
    source_path = os.path.join(original_address+ '/', path)
    destination_path = os.path.join(original_address +  '/Val/', path)
    shutil.move(source_path, destination_path)

In [86]:
train_data['data_path'] = train_data['path'].apply(lambda x: os.path.join('Train/',x))
val_data['data_path'] = val_data['path'].apply(lambda x: os.path.join('Val/',x))
test_data['data_path'] = test_data['path'].apply(lambda x: os.path.join('Test/',x))

In [87]:
new_train_df = train_data.drop(['index', 'path', 'patient'], axis = 1)
new_val_df = val_data.drop(['index', 'path', 'patient'], axis = 1)
new_test_df = test_data.drop(['index', 'path', 'patient'], axis = 1)

In [88]:
new_train_df.rename(columns={'data_path': 'path'}, inplace=True)
new_val_df.rename(columns={'data_path': 'path'}, inplace=True)
new_test_df.rename(columns={'data_path': 'path'}, inplace=True)

In [90]:
new_train_df.to_csv('train_labels_psax.csv', index=False)
new_val_df.to_csv('val_labels_psax.csv', index=False)
new_test_df.to_csv('test_labels_psax.csv', index=False)